In [1]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f"documents: {len(documents)}")

documents: 948


In [2]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, **index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

print("status")

  0%|          | 0/948 [00:00<?, ?it/s]

status


In [3]:
query = "How do I execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "multi_match": {
            "query": query,
            "fields": ["question^4", "text"], # Boost de 4 para el campo 'question'
            "type": "best_fields"
        }
    }
}

response = es_client.search(index=index_name, **search_query)

top_score = response['hits']['hits'][0]['_score']
print(f"score: {top_score}")

score: 55.260803


In [4]:
query = "How do I copy a file to a Docker container?"


search_query = {
    "size": 3,  
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp" 
                }
            }
        }
    }
}

response = es_client.search(index=index_name, **search_query)

results_q4 = response['hits']['hits']

print("questions:")
for hit in results_q4:
    print(f"- {hit['_source']['question']}")

questions:
- How do I debug a docker container?
- How do I debug a docker container?
- How do I debug a docker container?


In [5]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for hit in results_q4:
    doc = hit['_source']
    context += f"{context_template.format(question=doc['question'], text=doc['text'])}\n\n"

question_for_llm = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=question_for_llm, context=context)

prompt_length = len(prompt)
print(f"length: {prompt_length}")

length: 1324


In [6]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

num_tokens = len(tokens)

print(f"num_tok: {num_tokens}")

num_tok: 298


In [7]:
import os
import google.generativeai as genai

In [8]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [9]:
def llm(prompt):
    """
    """
    
    response = model.generate_content(prompt)
    
    return response.text

In [10]:
model = genai.GenerativeModel('gemini-1.5-flash-latest') 

print("Model Ok")

Model Ok


In [11]:
final_answer = llm(prompt)

print("Response generated by Gemini:")
print(final_answer)

Response generated by Gemini:
First, find the container ID using the command `docker ps`.  Then, execute the command `docker exec -it <container-id> bash` to execute a command in the specific container.



In [22]:
def llm(prompt):
    response = ollama.chat(
        model='tinyllama', 
        messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ]
    )
    return response['message']['content']

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=5, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')


In [24]:
index = TextSearch(
    text_fields=['section', 'question', 'text']
)

index.fit(documents)


In [25]:
CONTEXT_TEMPLATE = """
Q: {question}
A: {text}
""".strip()

PROMPT_TEMPLATE = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

def build_prompt(query, search_results):
    context = ""
    for doc in search_results:
        doc_str = CONTEXT_TEMPLATE.format(question=doc['question'], text=doc['text'])
        context += f"{doc_str}\n\n"
    
    prompt = PROMPT_TEMPLATE.format(
        question=query,
        context=context.strip()
    ).strip()
    
    return prompt

In [ ]:
query = 'how do I run kafka?'

search_results = index.search(query=query, n_results=5)

prompt = build_prompt(query, search_results)

answer = llm(prompt)

print("Response generated by tinyllama:")
print(answer)